In [96]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
nltk.download('punkt')
from scipy import spatial
import torch
import spacy
import PyPDF2 as PyPDF2
import tabula as tabula
en_nlp = spacy.load('en')

[nltk_data] Downloading package punkt to /home/dell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [97]:
import tika
tika.initVM()
from tika import parser
parsed = parser.from_file('data/book.pdf')
context = parsed["content"]

In [98]:
questions = []

questions.append('In Pillow, RGBA values are represented how ?')
questions.append('Normally transparent pixels are pasted how ?')
questions.append('Cropping an image means what ?')

In [99]:
contexts = []

for i in range(len(questions)):
    contexts.append(context)

In [100]:
df = pd.DataFrame({"context":contexts, "question": questions})

In [101]:
df.to_csv("data/train.csv", index = None)

In [102]:
blob = TextBlob(context)
sentences = [item.raw for item in blob.sentences]

In [103]:
len(sentences)

380

In [104]:
from models import InferSent

V = 1
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [105]:
W2V_PATH = 'GloVe/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [106]:
infersent.build_vocab(sentences, tokenize=True)

Found 1318(/1542) words with w2v vectors
Vocab size : 1318


In [107]:
dict_embeddings = {}
for i in range(len(sentences)):
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

In [108]:
for i in range(len(questions)):
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

In [109]:
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}

In [110]:
d1

{'# Check if width & height are larger than 500.\n        if TODO:\n            # Image is large enough to be considered a photo.': array([[ 0.08179848,  0.17447859,  0.12401593, ...,  0.01387853,
         -0.03814263,  0.02328598]], dtype=float32),
 '# TODO: Calculate the new width and height to resize to.': array([[ 0.07533996,  0.0851954 ,  0.13380973, ...,  0.01686598,
          0.0288246 , -0.0012792 ]], dtype=float32),
 '# TODO: Check if image needs to be resized.': array([[ 0.07533996,  0.09078287,  0.09210569, ..., -0.03236781,
          0.0166659 ,  0.00533651]], dtype=float32),
 '# TODO: Resize the image.': array([[0.08467202, 0.11090795, 0.10922538, ..., 0.02267995, 0.00782212,
         0.02519238]], dtype=float32),
 '(Or you can make it so that the values \nfor these constants are taken from the command line arguments.)': array([[ 0.08750732,  0.05459988,  0.07395696, ...,  0.06436399,
         -0.03814263,  0.05085603]], dtype=float32),
 '8.': array([[ 0.02435273,  0.11231

In [111]:
d2

{'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n17\nm A n i P u l A t i n g  i m A g e S\n\nIf you have a digital camera or even if \nyou just upload photos from your phone \n\nto Facebook, you probably cross paths with \ndigital image files all the time.': array([[ 0.04268799,  0.09025154,  0.12462997, ...,  0.02383171,
         -0.00577644,  0.10209721]], dtype=float32),
 '# TODO: Add the logo.': array([[ 0.0904335 ,  0.11682605,  0.10872702, ..., -0.00154413,
          0.05198823,  0.01415551]], dtype=float32),
 '# TODO: Save changes.': array([[ 0.07533996,  0.10101349,  0.06428269, ..., -0.02271397,
          0.04424981, -0.0242383 ]], dtype=float32),
 "'):\nv     if not (filename.endswith('.png') or filename.endswith('.jpg')) \\\n\n       or filename == LOGO_FILENAME:\nw         continue    # skip non-image files and the logo file itself\n\nx     im = Image.open(filename)\n    width, height = im.size\n\n--snip--\n\nFirst, the os.makedirs() call create

In [112]:
with open('data/dict_embeddings1.pickle', 'wb') as handle:
    pickle.dump(d1, handle)

In [113]:
with open('data/dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(d2, handle)

In [114]:
del dict_embeddings